In [22]:
import pathlib as pl
import pandas as pd
import re

pd.set_option('future.no_silent_downcasting', True)

%cd -q "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/notebooks"
_PROJECT_CONFIG_NB = str(pl.Path("00_project_config.ipynb").resolve(strict=True))

%run $_PROJECT_CONFIG_NB

_MYNAME="norm-qv-est-table-header"
_NBSTAMP=get_nb_stamp(_MYNAME)

ASSEMBLER = "verkko"

TABLE_FOLDER = PROJECT_DATA_ROOT.joinpath(
    "2024_merged_labels/summaries"
)

def norm_column_name(column_name):

    if column_name in ["sample", "assembler"]:
        return column_name
    else:
        label, suffix = column_name.split("_", 1)
        if label == "ASSM":
            label = "assembly"
        suffix_map = {
            "regions_uniq_pct": "singleton regions (%)",
            "single_label_bp": "singletons (bp)",
            "total_label_bp": "total (bp)",
            "single_regions_num": "singleton regions (n)",
            "total_regions_num": "total regions (n)",
            "total_regions_bp": "total regions (bp)",
            "total_gapless_length": "gapless length (bp)"
        }
        plain_text = suffix_map[suffix]
        return f"{label} {plain_text}"


for table_file in TABLE_FOLDER.glob("*ps*global-uniq.tsv"):
    print(table_file.name)
    df = pd.read_csv(table_file, sep="\t", header=0)

    drop_columns = [c for c in df.columns if "_merged_" in c]
    df.drop(drop_columns, axis=1, inplace=True)
    df["sample"] = df["sample"].apply(lambda s: s.split(".")[0])
    df.insert(1, "assembler", ASSEMBLER)
    assembly_columns = ["sample", "assembler", "ASSM_total_regions_num", "ASSM_total_regions_bp", "ASSM_total_gapless_length"]
    other_columns = sorted([c for c in df.columns if c not in assembly_columns])
    df = df[assembly_columns + other_columns]
    if "no-ont" in table_file.name:
        out_file = TABLE_OUT_SUPPL.joinpath(
            f"table_spe_{ASSEMBLER}_qc-label-summary.tsv"
        )
    else:
        # limit this table to just the overall assembly
        # and the Inspector ONT statistics. That table
        # is just the "why we ignored that"
        column_subset = [c for c in df.columns if any(key in c for key in ["sample", "assembler", "ASSM", "ISPCON"])]
        df = df[column_subset]
        out_file = TABLE_OUT_SUPPL.joinpath(
            f"table_spe_{ASSEMBLER}_inspector-ont-detail.tsv"
        )
    df.columns = [norm_column_name(c) for c in df.columns]
    df.to_csv(out_file, sep="\t", header=True, index=False)
        
        
    

SAMPLES.verkko.merged-issues.ps-no-ont.global-uniq.tsv
     sample assembler  ASSM_total_regions_num  ASSM_total_regions_bp  \
0   HG00096    verkko                   61526               34868587   
1   HG00171    verkko                   54545               34719966   
2   HG00268    verkko                   65927               46293010   
3   HG00358    verkko                   30275               31191256   
4   HG00512    verkko                   72000               52609111   
..      ...       ...                     ...                    ...   
60  NA20355    verkko                   71939              104735154   
61  NA20509    verkko                   47404               30482941   
62  NA20847    verkko                   81734               68785048   
63  NA21487    verkko                  100472               94644869   
64  NA24385    verkko                   32384               27390480   

    ASSM_total_gapless_length  BSCDUP_regions_uniq_pct  \
0                  584

RuntimeError: No active exception to reraise